In [5]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [282]:
url_inicial = 'http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/view/9'
url_root = 'http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/view/9'
r=requests.get(url_inicial)

In [283]:
soup = BeautifulSoup(r.text, 'html.parser')

In [284]:
lista_articulo = soup.find_all('div', class_='obj_article_summary')
links_libros = [x.find('a').get('href') for x in lista_articulo]
links_libros = [urljoin (url_root, i) for i in links_libros]
links_libros

['http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/129',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/130',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/131',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/132',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/133',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/134',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/135',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/136',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/137',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/138',
 'http://rmcg.geociencias.unam.mx/index.php/rmcg/article/view/139']

In [285]:
def get_url_items(soup,url):
    lista_articulo = soup.find_all('div', class_='obj_article_summary')
    links_libros = [x.find('a').get('href') for x in lista_articulo]
    links_libros = [urljoin (url_root, i) for i in links_libros]
    return links_libros

In [286]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('div.cmp_pagination > a')
    if not next_a or not next_a[0].get('href'):
        break
    url_inicial = urljoin(url_inicial, next_a[0].get('href'))

Estoy en la pagina http://rmcg.geociencias.unam.mx/index.php/rmcg/issue/view/9


In [287]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

11

In [288]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [289]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Revista Mexicana de Ciencias Geológicas'
    a='I. Físico Matemáticas y Ciencias de la Tierra'
    tem='Geociencias'
    url='http://rmcg.geociencias.unam.mx/'
    s_item=BeautifulSoup(r.text,'html.parser')
    #titulo de revista
    try:
        titulo=tire
        content_book['Titulo de revista']=tire
    except AttributeError:
        content_book['Titulo de revista']=None
    #area
    try:
        area=a
        content_book['Área']=a
    except AttributeError:
        content_book['Área']=None
    #tematica
    try:
        tema=tem
        content_book['Temática']=tem
    except AttributeError:
        content_book['Temática']=None
    #titulo libro
    try:
        titu=s_item.find('h1', class_='page_title').get_text(strip=True)
        content_book['Titulo del libro']=titu
    except AttributeError:
        content_book['Titulo del libro']=None
    #abstract
    try:
        abst=s_item.find('p').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #resumen
    try:
        resu=s_item.find('div', class_='description').get_text(strip=True)
        content_book['Resumen']=resu
    except AttributeError:
        content_book['Resumen']=None
    #Link incial
    try:
        inicial=url
        content_book['Link inicial']=url
    except AttributeError:
        content_book['Link inicial']=None
    #link articulo
    try:
        link=s_item.find('a', class_='obj_galley_link pdf').get('href')
        content_book['Link articulo']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link articulo']=None
    return content_book

In [290]:
list_scraper=list_scraper[0:24]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9
estas escrapeando la pag 10


In [291]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo de revista,Área,Temática,Titulo del libro,Abstract,Resumen,Link inicial,Link articulo
0,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,Preliminary paleoseismic results from the Past...,This work is licensed under aCreative Commons ...,None,http://rmcg.geociencias.unam.mx/,http://rmcg.geociencias.unam.mx/index.php/rmcg...
1,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,Genesis of manganese oxide mineralization in t...,Manganese oxide mineralization on the eastern ...,None,http://rmcg.geociencias.unam.mx/,http://rmcg.geociencias.unam.mx/index.php/rmcg...
2,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,Heat and mass fluxes monitoring of El Chichón ...,El Chichón crater lake is characterized by imp...,None,http://rmcg.geociencias.unam.mx/,http://rmcg.geociencias.unam.mx/index.php/rmcg...
3,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,Características microtexturales como indicador...,During its history Colima volcano has experien...,None,http://rmcg.geociencias.unam.mx/,http://rmcg.geociencias.unam.mx/index.php/rmcg...
4,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,Carbon and oxygen isotopic variations in early...,We used petrofacies analysis and stable isotop...,None,http://rmcg.geociencias.unam.mx/,http://rmcg.geociencias.unam.mx/index.php/rmcg...
5,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,Mixed evaporite/carbonate characteristics of t...,The Early Triassic Kangan Formation is a mixed...,None,http://rmcg.geociencias.unam.mx/,http://rmcg.geociencias.unam.mx/index.php/rmcg...
6,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,Dinámica de la acumulación sedimentaria de los...,The wedge top depozone overlies the frontal pa...,None,http://rmcg.geociencias.unam.mx/,http://rmcg.geociencias.unam.mx/index.php/rmcg...
7,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,Inventario y caracterización morfoclimática de...,This paper presents an inventory and a morphol...,None,http://rmcg.geociencias.unam.mx/,http://rmcg.geociencias.unam.mx/index.php/rmcg...
8,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,Geoquímica de tierras raras y fosfogénesis en ...,Major and rare earth elements (REE) in authige...,None,http://rmcg.geociencias.unam.mx/,http://rmcg.geociencias.unam.mx/index.php/rmcg...
9,Revista Mexicana de Ciencias Geológicas,I. Físico Matemáticas y Ciencias de la Tierra,Geociencias,Taxonomy and notes on the paleobiology of the ...,A set of cranial and dental material of Antilo...,None,http://rmcg.geociencias.unam.mx/,http://rmcg.geociencias.unam.mx/index.php/rmcg...


In [292]:
df_catalogo.to_csv('Revista15.26.csv', index=False)